# Imports and Setup

In [1]:
%load_ext autoreload
%autoreload 2
    
# Necessary/extra dependencies. 
import shutil

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

# ClearML
from clearml import Task
import dotenv
dotenv.load_dotenv()
!echo $CLEARML_API_ACCESS_KEY
!echo $CLEARML_API_SECRET_KEY

KW2GX2IN2YCKZLNTUSDH
OjO23n5avatAGjlAEkMjeyQrEZnW1ejOz600Z4JcKdcNttGDjO


Move `data.yaml` in the `yolov5/data` directory.

In [3]:
%ls

 data/                                     main_train.py
 dataset/                                  nohup.out
 data.yaml                                 README.dataset.txt
 Dockerfile                                README.md
 Dockerfile_jetson                         README.roboflow.txt
'folds - negative.v2i.yolov5pytorch.zip'   requirements_jetson.txt
 LICENSE                                   requirements.txt
 main_inference.py                         yolov5.ipynb


In [5]:
shutil.copy('./data/data.yaml', './yolov5/data')

'./yolov5/data/data.yaml'

# 2.Train

Train a YOLOv5s model on the custom dataset starting from pretrained `--weights yolov5s.pt`. Training results are saved to `runs/train/` with incrementing run directories, i.e. `runs/train/exp2`, `runs/train/exp3` etc.

In [6]:
%cd yolov5/data
%ls ../../data/train
%cd ..
%ls

/home/iseng/fmonti/cardboard_yolo/yolov5/data
images/  labels/


/home/iseng/miniconda3/envs/rota/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/iseng/fmonti/cardboard_yolo/yolov5
benchmarks.py    data/       LICENSE         README.zh-CN.md   tutorial.ipynb
CITATION.cff     detect.py   models/         requirements.txt  utils/
classify/        export.py   pyproject.toml  segment/          val.py
CONTRIBUTING.md  hubconf.py  README.md       train.py


In [21]:
task = Task.init(project_name='yolov5', task_name='dataset_fold_aug')

Jupyter Notebook auto-logging failed, could not access: /home/iseng/fmonti/cardboard_yolo/yolov5/yolov5.ipynb


ClearML Task: created new task id=c703913bf61447d18633b365dda38295
ClearML results page: https://app.clear.ml/projects/e9ac8f93b6aa41df88278a75e6c3909d/experiments/c703913bf61447d18633b365dda38295/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [22]:
!python3 train.py --img 640 --batch 64 --epochs 300 --data data/data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=data/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24252MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5

In [12]:
task.close()

# 3. Inference

We use the best weights for the inference

In [16]:
%ls

benchmarks.py    detect.py   __pycache__/      runs/           val.py
CITATION.cff     export.py   pyproject.toml    segment/        yolov5s.pt
classify/        hubconf.py  README.md         train.py
CONTRIBUTING.md  LICENSE     README.zh-CN.md   tutorial.ipynb
data/            models/     requirements.txt  utils/


In [14]:
!python3 detect.py --weights runs/train/exp2/weights/best.pt --img 640 --source ../data/test/images --line-thickness 1 --save-txt

detect: weights=['runs/train/exp2/weights/best.pt'], source=../data/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24252MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/175 /home/iseng/fmonti/cardboard_yolo/data/test/images/c1_51_b8_a0_61_d8_23_rot_jpeg.rf.a41c8fd7cc39994fd46a3e97a281549e.jpg: 640x640 44 folds, 3.3ms
image 2/175 /home/iseng/fmonti/cardboard_yolo/data/test/images/c1_51_b8_a0_61_d8_7_rot_jpeg.rf.1c5b9c130d1e1664e6d9a58443a50948.jpg: 640x640 45 folds, 3.7ms

In [15]:
#display inference on ALL test images
#this looks much better with longer training above

import glob
from IPython.display import Image, display

for imageName in glob.glob('runs/detect/exp/*.jpeg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")

# Test inference

In [20]:
!python3 detect.py --weights runs/train/exp2/weights/best.pt --img 640 --conf 0.6 --source ./../cartone_edit_rot.png --line-thickness 1 --save-txt

detect: weights=['runs/train/exp2/weights/best.pt'], source=./../cartone_edit_rot.png, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.6, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=1, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-321-g3742ab49 Python-3.10.14 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24252MiB)

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 /home/iseng/fmonti/cardboard_yolo/cartone_edit_rot.png: 640x640 2 folds, 3.2ms
Speed: 0.2ms pre-process, 3.2ms inference, 258.8ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp5
1 labels saved to runs/detect/exp5/labels
